In [ ]:
import CellModeller.CellState
import pickle
import numpy as np
import matplotlib.pyplot as plt
from random import random
import os

In [ ]:
fname = '/Users/timrudge/cellmodeller/data/ex5_colonySector-20-04-21-14-10/step-00740.pickle'
data = pickle.load(open(fname, 'rb'))

In [ ]:
cs = data['cellStates']
lineage = data['lineage']
len(cs)

In [ ]:
counts = {}
counts_random = {}
N = [0,0]
for id1,cell1 in cs.items():
    p1 = cell1.pos
    parent1 = lineage[cell1.id]
    for id2,cell2 in cs.items():
        type1 = 1 * (cell1.n_a > cell1.n_b)
        type2 = 1 * (cell2.n_a > cell2.n_b)
        N[type1] += 1
        N[type2] += 1
        p2 = cell2.pos
        parent2 = lineage[cell2.id]
        d = np.sqrt( (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 )
        idx = int(d//5)
        if type1!=type2:
            if idx in counts:
                counts[idx] += 1
            else:
                counts[idx] = 1
        if random()<0.5:
            if idx in counts_random:
                counts_random[idx] += 1
            else:
                counts_random[idx] = 1

In [ ]:
plt.plot(list(counts.keys()), list(counts.values()), 'r.')
print(counts.keys())

plt.plot(list(counts_random.keys()), list(counts_random.values()), 'k.')
print(counts_random.keys())

In [ ]:
ratio = []
for i in range(10):
    ratio.append(counts[i]/counts_random[i] -1)
plt.plot(ratio)

In [ ]:
def relatedness(id1, id2, lineage):
    p1 = id1
    p2 = id2
    path2 = 0
    steps = 0
    
    # Find common ancestor
    while p1!=p2 and p1!=1 and p2!=1:
        p1 = lineage.get(p1, 1)
        p2 = lineage.get(p2, 1)
    if p1==p2:
        ancestor = p1
    else:
        ancestor = 1
        
    # Find path length from cell1 to ancestor
    path1 = 0
    p1 = id1
    while p1!=ancestor:
        p1 = lineage.get(p1,1)
        path1 += 1
        
    # Find path length from cell2 to ancestor
    path2 = 0
    p2 = id2
    while p2!=ancestor:
        p2 = lineage.get(p2,1)
        path2 += 1
        
    rel = 2**(-path1-path2)
    return rel

In [ ]:
relatedness(10, 3, lineage)

In [ ]:
dirs = [
    'ex5_colonySector-20-04-21-14-10',
    'ex5_colonySector-20-04-21-14-14',
    'ex5_colonySector-20-04-21-14-16',
    'ex5_colonySector-20-04-21-14-16_2',
    'ex5_colonySector-20-04-21-14-22',
    'ex5_colonySector-20-04-21-14-22_2'
]

counts = {}
sum_rel = {}

for d in dirs:
    fname = os.path.join('/Users/timrudge/cellmodeller/data', d, 'step-00740.pickle')
    data = pickle.load(open(fname, 'rb'))
    cs = data['cellStates']
    lineage = data['lineage']
    for id1,cell1 in cs.items():
        p1 = cell1.pos
        for id2,cell2 in cs.items():
            if id1!=id2:
                p2 = cell2.pos
                d = np.sqrt( (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 )
                idx = int(d//5)
                if idx in counts:
                    counts[idx] += 1
                else:
                    counts[idx] = 1
                rel = relatedness(id1, id2, lineage)
                if idx in sum_rel:
                    sum_rel[idx] += rel
                else:
                    sum_rel[idx] = rel


In [ ]:
counts_r = np.array(list(counts.keys())) * 5.
sum_rel_r = np.array(list(sum_rel.keys())) * 5.

counts = np.array(list(counts.values()))
sum_rel = np.array(list(sum_rel.values()))

In [ ]:
plt.plot(counts_r, counts, 'r.')
plt.figure()
plt.plot(sum_rel_r, sum_rel, 'r.')
plt.figure()
plt.plot(sum_rel_r, sum_rel/counts, 'k.')
plt.figure()
plt.plot(sum_rel_r, np.log(sum_rel/counts), 'k.')
plt.figure()
plt.plot(np.log(sum_rel_r+2.5), np.log(sum_rel/counts), 'k.')

In [ ]:
x = np.log(sum_rel_r+2.5)
y = np.log(sum_rel/counts)

z = np.polyfit(x, y, deg=1)
print(z)

plt.plot(sum_rel_r, np.log(sum_rel_r)*z[0] + z[1], 'r.')
plt.plot(sum_rel_r, np.log(sum_rel/counts), 'k.')

plt.figure()
plt.plot(np.log(sum_rel_r+2.5), np.log(sum_rel_r)*z[0] + z[1], 'r.')
plt.plot(np.log(sum_rel_r+2.5), np.log(sum_rel/counts), 'k.')

In [ ]:
np.any(~np.isfinite(sum_rel/counts))

In [ ]:
# Compute Moran's I
dirs = [
    'ex5_colonySector-20-04-21-14-10',
    #'ex5_colonySector-20-04-21-14-14',
    #'ex5_colonySector-20-04-21-14-16',
    #'ex5_colonySector-20-04-21-14-16_2',
    #'ex5_colonySector-20-04-21-14-22',
    #'ex5_colonySector-20-04-21-14-22_2'
]

Is = []
for d in dirs:
    fname = os.path.join('/Users/timrudge/cellmodeller/data', d, 'step-00340.pickle')
    data = pickle.load(open(fname, 'rb'))
    cs = data['cellStates']

    # Compute mean x where x=n_a number of plasmid type a
    n_a = [cell.n_a for id,cell in cs.items()]
    x_mean = np.mean(n_a)

    # Compute numerator and denominator of Moran's I
    sum_dev_x1 = 0
    sum_codev = 0
    W = 0
    for id1,cell1 in cs.items():
        p1 = cell1.pos
        x1 = cell1.n_a
        sum_dev_x1 += (x1-x_mean)**2
        for id2,cell2 in cs.items():
            if id1!=id2:
                p2 = cell2.pos
                x2 = cell2.n_a
                d = np.sqrt( (p1[0]-p2[0])**2 + (p1[1]-p2[1])**2 )
                w12 = 1/d/d
                W += w12
                sum_codev += w12 * (x1-x_mean) * (x2-x_mean)
    N = len(cs)
    I = N/W * sum_codev / sum_dev_x1
    Is.append(I)

In [ ]:
print(x_mean)
print(np.sqrt(sum_dev_x1/N))
print(Is)